## Downloading the data

The dataset is retrieved from the website http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz using as a reference the following research paper


> Lukas Bossard, Matthieu Guillaumin, Luc Van Gool - Food-101 – Mining Discriminative Components with Random Forests

The Food-101 data set consists of images from Foodspotting [1]. Any use beyond
   scientific fair use must be negociated with the respective picture owners
   according to the Foodspotting terms of use [2].

[1] http://www.foodspotting.com/
[2] http://www.foodspotting.com/terms/

In [1]:
#importing essential modules
import pandas as pd
import numpy as np
import os
from os import path
import time
from random import seed, choice
import shutil

In [2]:
#deleting the data folder in case a cleaning is required
#shutil.rmtree("../data")

In [3]:
#the data will be downloaded and automatically extracted to the data folder ../data/food-101/images
%mkdir ../data
!wget -O ../data/food-101.tar.gz http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -zxf ../data/food-101.tar.gz -C ../data

--2020-03-03 07:27:14--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2020-03-03 07:27:14--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘../data/food-101.tar.gz’

../data/food-101.ta 100%[===================>]   4.65G  18.2MB/s    in 4m 37s  

2020-03-03 07:31:52 (17.2 MB/s) - ‘../data/food-101.tar.gz’ saved [4996278331/4996278331]



## Organise train and test set

In [2]:
#dividing into train and test set using the json metadata 

metafolder = "../data/food-101/meta/"
train_meta = pd.read_json(path_or_buf = metafolder + "train.json")
test_meta = pd.read_json(path_or_buf = metafolder + "test.json")

> ### Organising metdatada for training, testing and validation

In [3]:
#organising metadata for training, testing and validation
validation_split = 0.2
val_split_idx = int(np.floor(train_meta.shape[0]*validation_split))

#folder with all the food images
data_dir = "../data/food-101/images/"
folders_sorted = sorted(os.listdir(data_dir))

#number of categories to randomly select
nc = 5

#selecting a randomn subset of categories
seed(42)

selection = []
while len(selection) < nc:
    pick = choice(folders_sorted)
    if pick not in set(selection):
        selection.append(pick)
        
print("Selected categories : {}".format(', '.join(map(str, selection))))

Selected categories : ramen, carrot_cake, beef_carpaccio, strawberry_shortcake, escargots


In [4]:
#create folder with data to upload to s3
%mkdir ../data/s3_train_data
%mkdir ../data/s3_train_data/train_img 
%mkdir ../data/s3_train_data/valid_img
%mkdir ../data/test_img

train_meta = train_meta[selection].iloc[:train_meta.shape[0] - val_split_idx]

valid_meta = train_meta[selection].iloc[train_meta.shape[0] - val_split_idx:]

test_meta = test_meta[selection]

#Setting train, validation and test set target folder
#target folder - train
trainfolder = "../data/s3_train_data/train_img/"

#target folder - validation
validfolder = "../data/s3_train_data/valid_img/"

#target folder -test
testfolder = "../data/test_img/"

print("{} images used for training".format(train_meta.shape[0]*train_meta.shape[1]))
print("{} images used for validation".format(valid_meta.shape[0]*valid_meta.shape[1]))
print("{} images used for testing".format(test_meta.shape[0]*test_meta.shape[1]))

mkdir: cannot create directory ‘../data/s3_train_data’: File exists
mkdir: cannot create directory ‘../data/s3_train_data/train_img’: File exists
mkdir: cannot create directory ‘../data/s3_train_data/valid_img’: File exists
mkdir: cannot create directory ‘../data/test_img’: File exists
3000 images used for training
750 images used for validation
1250 images used for testing


In [7]:
#dividing into train and test set using the json metadata 

def organise_files_from_df(df, datafolder, datatarget):
    """
    This function moves files contained in a folder (datafolder) to a target path (datatarget),
    based on the information contained on a dataframe (df) where each column corresponds to a 
    class name (sub-folder). Every column of the dataset contains a list of filenames to be moved.
    """
    
    #creating target folder
    if not path.exists(datatarget):
        os.mkdir(datatarget)
    
    #iterating through dataframe columns ( =  labels)
    for label in list(df.columns):
        
        #create folder
        foldername = datatarget + str(label)
        
        if not path.exists(foldername):
            os.mkdir(foldername)
        
        #move each file
        for file in list(df[label]):
            
            fileoriginal =  datafolder + file + ".jpg"
            filetarget = datatarget +"/" + file + ".jpg"
            
            try:
                if not path.exists(filetarget):
                    shutil.copyfile(fileoriginal, filetarget)

            except FileNotFoundError:
                print("File {} not found!".format(file))
                pass

#origin folder
imagefolder = "../data/food-101/images/"


organise_files_from_df(train_meta, imagefolder, trainfolder)

organise_files_from_df(valid_meta, imagefolder, validfolder)

organise_files_from_df(test_meta, imagefolder, testfolder)

#to delete the origin folder, uncomment the line below
#shutil.rmtree(imagefolder)

## Load Data to S3

>The below cells load in some AWS SageMaker libraries, starts a SageMaker session and creates a default bucket. After creating this bucket, it upload the locally stored data to S3.

In [5]:
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

> ### Upload training and testing data

In [6]:
prefix = "food-classifier"
datafolder = "../data/s3_train_data"
# upload all data to S3

#this is slow!
start = time.time()
input_data = sagemaker_session.upload_data(path=datafolder, bucket=bucket, key_prefix=prefix)
end = time.time()

print("Data uploaded to s3 after {} seconds".format(end - start))

Data uploaded to s3 after 293.68911957740784 seconds


In [10]:
# check that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    #print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('All good!')

All good!


> ## Checking model.py

In [7]:
!pygmentize pytorch_source/model.py

import torch
import torchvision.models as models

#importing pretrained ResNet for transfer learning
ResNetTransfer = models.resnet50(pretrained=True) #.load_state_dict(torch.load("resnet50_base.pt"))


> ## Checking train.py

In [8]:
!pygmentize pytorch_source/train.py

import argparse
import json
import os
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data

from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# imports the model in model.py by name
from model import ResNetTransfer


def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device {}.".format(device))

    model = ResNetTransfer

    #fre

> ## Create pytorch estimator

In [9]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='pytorch_source', 
                    role=role,
                    framework_version= '1.1.0', #'1.3.1',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'n_classes': nc + 1,  # num of classes for the fully connected layer at the end of the network (defined on the first cells)
                        'n_epochs': 5,
                        'img_short_side_resize':256,
                        'img_input_size':224,
                        'num_workers':16,
                        'batch_size':64
                    })

In [10]:
estimator.fit({'training': input_data})

2020-03-03 15:54:05 Starting - Starting the training job...
2020-03-03 15:54:07 Starting - Launching requested ML instances......
2020-03-03 15:55:05 Starting - Insufficient capacity error from EC2 while launching instances, retrying!......
2020-03-03 15:56:07 Starting - Preparing the instances for training......
2020-03-03 15:57:31 Downloading - Downloading input data......
2020-03-03 15:58:25 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-03 15:58:46,303 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-03 15:58:46,330 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-03 15:58:46,959 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-03 15:58:48,904 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-03

> ## Deploy the model for testing

The model will be tested by first deploying it and then sending the testing data to the deployed endpoint

The function that loads the saved model is called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file specified as the entry point.

In [11]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.p2.xlarge")

-----------------------------------------------------------------------------------------------!

> ### Transformers set-up for test data

In [12]:
#Norm values
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

#Img size parameters
img_short_side_resize = 256
img_input_size = 224

import os
from torchvision import datasets
import torchvision.transforms as transforms

transform_test = transforms.Compose([
                    transforms.Resize(img_input_size),  
                    transforms.FiveCrop(img_input_size),
                    transforms.Lambda(lambda crops: torch.stack([transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(mean = norm_mean, std = norm_std)])(crop) for crop in crops]))])

test_data = datasets.ImageFolder(testfolder, transform_test)

> ### Data loaders

In [13]:
import torch

shuffle = True
num_workers = 16
batch_size = 64

test_loader = torch.utils.data.DataLoader(test_data, batch_size=int(np.floor(batch_size/5)), num_workers=0, shuffle=shuffle, pin_memory=True)

> ### Testing function implementation

In [22]:
def test(loader, model, criterion, device):
    """
    test function
    """
    
    test_loss = 0.
    correct = 0.
    total = 0.
    
    #model.eval()
    
    with torch.no_grad():
        
        for batch_idx, (data, target) in enumerate(loader):
            
            data, target = data.to(device), target.to(device) # move to GPU
            
            bs, ncrops, c, h, w = data.size()
            
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model.predict(data.view(-1, c, h, w)) # fuse batch size and ncrops
            output = output.view(bs, ncrops, -1).mean(1)    
            
            loss = criterion(output, target) # calculate the loss
            
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss)) # update average test loss 
            
            pred = output.data.max(1, keepdim=True)[1] # convert output probabilities to predicted class
            
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy()) 
            total += data.size(0)            
            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

> ## Testing

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
print("Using device {}.".format(device))
if torch.cuda.is_available():
    print("Using",torch.cuda.get_device_name(device))

Using device cpu.


In [ ]:
predictor.predict(test_data)

In [24]:
test(test_loader, predictor, torch.nn.CrossEntropyLoss(), device)

ConnectionClosedError: Connection was closed before we received a valid response from endpoint URL: "https://runtime.sagemaker.eu-central-1.amazonaws.com/endpoints/sagemaker-pytorch-2020-03-03-15-53-46-306/invocations".

> ## Delete endpoint

In [ ]:
estimator.delete_endpoint()